# Data Wrangling for TA12 IE Project
* Combine multiple sheets into single dataframe
* Convert data from wide format to long format

In [1]:
import pandas as pd

In [2]:
# Read excel file
xls = pd.ExcelFile('Moving annual rent by suburb - September quarter 2022.xlsx') 

In [3]:
# Create list of sheet names
print(xls.sheet_names)

['1 bedroom flat', '2 bedroom flat', '3 bedroom flat', '2 bedroom house', '3 bedroom house', '4 bedroom house', 'All properties']


In [4]:
# Read each sheet in excel file into individual dataframes
df_1f = pd.read_excel(xls, '1 bedroom flat')
df_2f = pd.read_excel(xls, '2 bedroom flat')
df_3f = pd.read_excel(xls, '3 bedroom flat')
df_2h = pd.read_excel(xls, '2 bedroom house')
df_3h = pd.read_excel(xls, '3 bedroom house')
df_4h = pd.read_excel(xls, '4 bedroom house')
df_all = pd.read_excel(xls, 'All properties')

In [5]:
# Create a list of all dataframes
df_list = [df_1f,df_2f,df_2h,df_3f,df_3h,df_4h,df_all]

In [6]:
processed_df_list = []
for dataframe in df_list:
    
    # Combining text of header rows 
    for i in range(2,len(dataframe.loc[0])):
        if pd.isnull(dataframe.loc[0][i]):
            dataframe.loc[0][i] = str(dataframe.loc[0][i-1]) +','+ str(dataframe.loc[1][i])+','+str(dataframe.loc[0][0])
            dataframe.loc[0][i-1] = str(dataframe.loc[0][i-1]) +','+ str(dataframe.loc[1][i-1])+','+str(dataframe.loc[0][0])
    
    # Changing the headers of the first 2 columns
    dataframe.iat[0,0] = 'Region' 
    dataframe.iat[0,1] = 'Suburb' 
    
    # Drop redundant row containing old header name
    dataframe = dataframe.drop(1)
    
    # Changing column names to row[0]
    dataframe = dataframe.rename(columns=dataframe.iloc[0]).drop(0).reset_index(drop=True)
    
    # Create a loop to replace NaN values in Region column with names of regions
    label = ''
    for i in range(len(dataframe['Region'])):
        if pd.notnull(dataframe['Region'][i]):
            label=str(dataframe['Region'][i])
        elif pd.isnull(dataframe['Region'][i]):
            dataframe['Region'][i] = label
    
    # Changing format of dataframe from wide to long
    dataframe = pd.melt(dataframe, col_level=0, id_vars=['Region','Suburb'])
    
    # Renaming variable column to Month_year and value column to Value
    dataframe = dataframe.rename(columns={'variable':'Month','value':'Value'})
    
    # Split Month column into 3 separate columns with Month data, Measure data and Housing Type data
    dataframe[['Month','Measure','Housing_Type']] = dataframe['Month'].str.split(',',expand=True)
    
    # Split Month column into 2 separate columns with Month data and Year data
    dataframe[['Month','Year']] = dataframe['Month'].str.split(' ',expand=True)
    
    processed_df_list.append(dataframe)


df = pd.concat(processed_df_list)
df.to_csv('rental_data.csv')

In [7]:
df.head()

,Region,Suburb,Month,Value,Measure,Housing_Type,Year
0,Inner Melbourne,Albert Park-Middle Park-West St Kilda,Mar,352,Count,1 bedroom flats,2000
1,Inner Melbourne,Armadale,Mar,210,Count,1 bedroom flats,2000
2,Inner Melbourne,Carlton North,Mar,87,Count,1 bedroom flats,2000
3,Inner Melbourne,Carlton-Parkville,Mar,303,Count,1 bedroom flats,2000
4,Inner Melbourne,CBD-St Kilda Rd,Mar,755,Count,1 bedroom flats,2000
